In [11]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import re                                  # library for regular expression operations
import string                              # for string operations
import numpy as np
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming 
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

# 1 - preparations de nos données

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\stive\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [54]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

x_train = train_pos + train_neg 
x_test = test_pos + test_neg

In [8]:
# combine positive and negative labels
y_train = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
y_test = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [9]:
# Print the shape train and test sets
print("y_train.shape = " + str(y_train.shape))
print("y_test.shape = " + str(y_test.shape))

y_train.shape = (8000, 1)
y_test.shape = (2000, 1)


# 2 - definissons nos fonctions

In [12]:
def process_tweet(tweet):
    """
        Input : 
            tweet : est une liste de tweet brut
        Output:
            array of word : tokenizes the tweet into individual words, removes stop words and applies stemming
    """
    # remove old style retweet text "RT"
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet2 = re.sub(r'#', '', tweet2)
    
    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                   reduce_len=True)

    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet2)
    
    stopwords_english = stopwords.words('english') 
    
    tweets_clean = []

    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
            
    # Instantiate stemming class
    stemmer = PorterStemmer()

    # Create an empty list to store the stems
    tweets_stem = [] 

    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list
        
    return tweets_stem


In [40]:
def build_freqs(text,label):
    """
        cette fonction va nous permettre de compter la frequence de chaque mot dans nos tweet 
        selon qu'il est utilisé dans un tweet positif ou dans un tweet négatif
    """
    freqs = {}
    
    for y,tweet in zip(label, text):
        
        for word in process_tweet(tweet):
            pair = (word, y)
           
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [14]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: un vecteur de taille(1,3)
            qui va compter le nombre de mot positif/négatif du tweet biensur en tenant compte de leur poids
            dans le dictionnaire freqs
        
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)
        

    return x

In [15]:
def sigmoid(z): 
    return 1/(1+np.exp(-1*z))


In [16]:

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: final weight vector
    '''

    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = x@theta
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = -1.0/m*((y.T)@np.log(h) + ((1-y).T)@np.log(1-h))
        print("epochs {}/{} ======> loss : {}\n".format(i,num_iters,J[0,0]))

        # update the weights theta
        theta = theta - (alpha/m)*(x.T@(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [17]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''

    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(x@theta)
    
    return y_pred

# 3 - testons nos fonctions

In [22]:
#example

tweet = x_train[0]
result = process_tweet(tweet)
print("Notre tweet initial:\n{}\n\nNotre tweet après traitement:\n{}".format(tweet,result))

Notre tweet initial:
#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Notre tweet après traitement:
['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [23]:
np.random.seed(1)
# X c'est une matrice 10*3 avec la premiere colone qui est le biais
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

epochs 0/700 ======> loss : 0.6931471805599453

epochs 1/700 ======> loss : 0.69305407419931

epochs 2/700 ======> loss : 0.6929615149729415

epochs 3/700 ======> loss : 0.6928694989001035

epochs 4/700 ======> loss : 0.6927780220289215

epochs 5/700 ======> loss : 0.6926870804361839

epochs 6/700 ======> loss : 0.6925966702271453

epochs 7/700 ======> loss : 0.6925067875353309

epochs 8/700 ======> loss : 0.692417428522341

epochs 9/700 ======> loss : 0.6923285893776581

epochs 10/700 ======> loss : 0.6922402663184535

epochs 11/700 ======> loss : 0.6921524555893973

epochs 12/700 ======> loss : 0.6920651534624666

epochs 13/700 ======> loss : 0.6919783562367572

epochs 14/700 ======> loss : 0.6918920602382954

epochs 15/700 ======> loss : 0.691806261819851

epochs 16/700 ======> loss : 0.6917209573607516

epochs 17/700 ======> loss : 0.691636143266698

epochs 18/700 ======> loss : 0.6915518159695805

epochs 19/700 ======> loss : 0.6914679719272961

epochs 20/700 ======> loss : 0.6913

In [41]:
# create frequency dictionary
freqs = build_freqs(x_train,y_train.reshape(8000,))

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11339


## 3.1 - extractions des features

In [47]:
# x va contenir nos différentes features
# cad pour chaque tweet nous alons compter
x = np.zeros((len(x_train), 3))
for i in range(len(x_train)):
    x[i, :]= extract_features(x_train[i], freqs)


y = y_train

# 4 - entrainons notre modèles

In [48]:

# Apply gradient descent
J, theta = gradientDescent(x, y, np.zeros((3, 1)), 1e-9,3000)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

epochs 0/3000 ======> loss : 0.6931471805599468

epochs 1/3000 ======> loss : 0.6920758566137005

epochs 2/3000 ======> loss : 0.6910078259206593

epochs 3/3000 ======> loss : 0.6899430772294411

epochs 4/3000 ======> loss : 0.6888815992910963

epochs 5/3000 ======> loss : 0.6878233808597424

epochs 6/3000 ======> loss : 0.6867684106932008

epochs 7/3000 ======> loss : 0.685716677553633

epochs 8/3000 ======> loss : 0.6846681702081743

epochs 9/3000 ======> loss : 0.6836228774295654

epochs 10/3000 ======> loss : 0.6825807879967867

epochs 11/3000 ======> loss : 0.6815418906956847

epochs 12/3000 ======> loss : 0.6805061743195978

epochs 13/3000 ======> loss : 0.6794736276699833

epochs 14/3000 ======> loss : 0.6784442395570338

epochs 15/3000 ======> loss : 0.6774179988002984

epochs 16/3000 ======> loss : 0.676394894229293

epochs 17/3000 ======> loss : 0.6753749146841129

epochs 18/3000 ======> loss : 0.6743580490160364

epochs 19/3000 ======> loss : 0.673344286088127

epochs 20/300

epochs 236/3000 ======> loss : 0.5100590201562274

epochs 237/3000 ======> loss : 0.5095048662211404

epochs 238/3000 ======> loss : 0.5089520764505221

epochs 239/3000 ======> loss : 0.5084006460543249

epochs 240/3000 ======> loss : 0.5078505702637497

epochs 241/3000 ======> loss : 0.5073018443311391

epochs 242/3000 ======> loss : 0.5067544635298717

epochs 243/3000 ======> loss : 0.5062084231542562

epochs 244/3000 ======> loss : 0.5056637185194275

epochs 245/3000 ======> loss : 0.5051203449612415

epochs 246/3000 ======> loss : 0.5045782978361713

epochs 247/3000 ======> loss : 0.5040375725212048

epochs 248/3000 ======> loss : 0.5034981644137411

epochs 249/3000 ======> loss : 0.5029600689314863

epochs 250/3000 ======> loss : 0.502423281512355

epochs 251/3000 ======> loss : 0.5018877976143663

epochs 252/3000 ======> loss : 0.501353612715543

epochs 253/3000 ======> loss : 0.5008207223138115

epochs 254/3000 ======> loss : 0.5002891219269014

epochs 255/3000 ======> loss : 0.

epochs 437/3000 ======> loss : 0.4208719131084207

epochs 438/3000 ======> loss : 0.42051816132781594

epochs 439/3000 ======> loss : 0.4201651253582474

epochs 440/3000 ======> loss : 0.4198128031116155

epochs 441/3000 ======> loss : 0.41946119250773656

epochs 442/3000 ======> loss : 0.4191102914743071

epochs 443/3000 ======> loss : 0.4187600979468659

epochs 444/3000 ======> loss : 0.4184106098687592

epochs 445/3000 ======> loss : 0.4180618251911035

epochs 446/3000 ======> loss : 0.41771374187275057

epochs 447/3000 ======> loss : 0.41736635788025145

epochs 448/3000 ======> loss : 0.4170196711878209

epochs 449/3000 ======> loss : 0.4166736797773022

epochs 450/3000 ======> loss : 0.4163283816381322

epochs 451/3000 ======> loss : 0.4159837747673069

epochs 452/3000 ======> loss : 0.415639857169345

epochs 453/3000 ======> loss : 0.41529662685625574

epochs 454/3000 ======> loss : 0.4149540818475026

epochs 455/3000 ======> loss : 0.41461222016997057

epochs 456/3000 ======> lo

epochs 651/3000 ======> loss : 0.3586826242915954

epochs 652/3000 ======> loss : 0.35844468308094635

epochs 653/3000 ======> loss : 0.35820714554517025

epochs 654/3000 ======> loss : 0.35797001069403656

epochs 655/3000 ======> loss : 0.357733277540479

epochs 656/3000 ======> loss : 0.3574969451005824

epochs 657/3000 ======> loss : 0.3572610123935702

epochs 658/3000 ======> loss : 0.35702547844179316

epochs 659/3000 ======> loss : 0.35679034227071466

epochs 660/3000 ======> loss : 0.35655560290890204

epochs 661/3000 ======> loss : 0.35632125938801057

epochs 662/3000 ======> loss : 0.3560873107427745

epochs 663/3000 ======> loss : 0.35585375601099273

epochs 664/3000 ======> loss : 0.35562059423351866

epochs 665/3000 ======> loss : 0.35538782445424666

epochs 666/3000 ======> loss : 0.3551554457201008

epochs 667/3000 ======> loss : 0.3549234570810238

epochs 668/3000 ======> loss : 0.35469185758996286

epochs 669/3000 ======> loss : 0.3544606463028615

epochs 670/3000 =====

epochs 898/3000 ======> loss : 0.3101200105067595

epochs 899/3000 ======> loss : 0.30995800714597216

epochs 900/3000 ======> loss : 0.309796235027618

epochs 901/3000 ======> loss : 0.3096346936722423

epochs 902/3000 ======> loss : 0.30947338260168294

epochs 903/3000 ======> loss : 0.3093123013390665

epochs 904/3000 ======> loss : 0.309151449408803

epochs 905/3000 ======> loss : 0.30899082633658254

epochs 906/3000 ======> loss : 0.30883043164937013

epochs 907/3000 ======> loss : 0.3086702648754027

epochs 908/3000 ======> loss : 0.30851032554418406

epochs 909/3000 ======> loss : 0.3083506131864806

epochs 910/3000 ======> loss : 0.3081911273343174

epochs 911/3000 ======> loss : 0.3080318675209743

epochs 912/3000 ======> loss : 0.3078728332809816

epochs 913/3000 ======> loss : 0.30771402415011495

epochs 914/3000 ======> loss : 0.3075554396653932

epochs 915/3000 ======> loss : 0.3073970793650725

epochs 916/3000 ======> loss : 0.3072389427886437

epochs 917/3000 ======> los

...

epochs 1860/3000 ======> loss : 0.21820941173504052

epochs 1861/3000 ======> loss : 0.21815343619380914

epochs 1862/3000 ======> loss : 0.21809750907931794

epochs 1863/3000 ======> loss : 0.21804163032898596

epochs 1864/3000 ======> loss : 0.2179857998803384

epochs 1865/3000 ======> loss : 0.2179300176710059

epochs 1866/3000 ======> loss : 0.2178742836387244

epochs 1867/3000 ======> loss : 0.21781859772133477

epochs 1868/3000 ======> loss : 0.21776295985678343

epochs 1869/3000 ======> loss : 0.2177073699831209

epochs 1870/3000 ======> loss : 0.2176518280385021

epochs 1871/3000 ======> loss : 0.21759633396118694

epochs 1872/3000 ======> loss : 0.2175408876895386

epochs 1873/3000 ======> loss : 0.21748548916202443

epochs 1874/3000 ======> loss : 0.21743013831721525

epochs 1875/3000 ======> loss : 0.21737483509378538

epochs 1876/3000 ======> loss : 0.21731957943051253

epochs 1877/3000 ======> loss : 0.2172643712662769

epochs 1878/3000 ======> loss : 0.2172092105400618

e

epochs 2102/3000 ======> loss : 0.20594292636878384

epochs 2103/3000 ======> loss : 0.20589706456282866

epochs 2104/3000 ======> loss : 0.2058512386776094

epochs 2105/3000 ======> loss : 0.20580544867081346

epochs 2106/3000 ======> loss : 0.20575969450019418

epochs 2107/3000 ======> loss : 0.20571397612356976

epochs 2108/3000 ======> loss : 0.20566829349882348

epochs 2109/3000 ======> loss : 0.2056226465839039

epochs 2110/3000 ======> loss : 0.20557703533682434

epochs 2111/3000 ======> loss : 0.2055314597156625

epochs 2112/3000 ======> loss : 0.20548591967856136

epochs 2113/3000 ======> loss : 0.2054404151837282

epochs 2114/3000 ======> loss : 0.20539494618943457

epochs 2115/3000 ======> loss : 0.2053495126540163

epochs 2116/3000 ======> loss : 0.20530411453587324

epochs 2117/3000 ======> loss : 0.20525875179346967

epochs 2118/3000 ======> loss : 0.20521342438533346

epochs 2119/3000 ======> loss : 0.2051681322700563

epochs 2120/3000 ======> loss : 0.20512287540629362


epochs 2344/3000 ======> loss : 0.1958015002691954

epochs 2345/3000 ======> loss : 0.19576323429268538

epochs 2346/3000 ======> loss : 0.19572499564860318

epochs 2347/3000 ======> loss : 0.19568678430739833

epochs 2348/3000 ======> loss : 0.1956486002395622

epochs 2349/3000 ======> loss : 0.19561044341562792

epochs 2350/3000 ======> loss : 0.19557231380617082

epochs 2351/3000 ======> loss : 0.19553421138180777

epochs 2352/3000 ======> loss : 0.19549613611319763

epochs 2353/3000 ======> loss : 0.19545808797104064

epochs 2354/3000 ======> loss : 0.1954200669260789

epochs 2355/3000 ======> loss : 0.19538207294909563

epochs 2356/3000 ======> loss : 0.195344106010916

epochs 2357/3000 ======> loss : 0.19530616608240622

epochs 2358/3000 ======> loss : 0.19526825313447382

epochs 2359/3000 ======> loss : 0.1952303671380676

epochs 2360/3000 ======> loss : 0.19519250806417762

epochs 2361/3000 ======> loss : 0.19515467588383478

epochs 2362/3000 ======> loss : 0.1951168705681111



epochs 2558/3000 ======> loss : 0.1881912467305609

epochs 2559/3000 ======> loss : 0.1881582183358538

epochs 2560/3000 ======> loss : 0.18812521180072772

epochs 2561/3000 ======> loss : 0.18809222710316562

epochs 2562/3000 ======> loss : 0.18805926422117972

epochs 2563/3000 ======> loss : 0.1880263231328111

epochs 2564/3000 ======> loss : 0.18799340381613017

epochs 2565/3000 ======> loss : 0.18796050624923666

epochs 2566/3000 ======> loss : 0.18792763041025892

epochs 2567/3000 ======> loss : 0.18789477627735476

epochs 2568/3000 ======> loss : 0.1878619438287105

epochs 2569/3000 ======> loss : 0.18782913304254176

epochs 2570/3000 ======> loss : 0.1877963438970926

epochs 2571/3000 ======> loss : 0.18776357637063643

epochs 2572/3000 ======> loss : 0.1877308304414749

epochs 2573/3000 ======> loss : 0.18769810608793844

epochs 2574/3000 ======> loss : 0.18766540328838663

epochs 2575/3000 ======> loss : 0.18763272202120715

epochs 2576/3000 ======> loss : 0.18760006226481651


epochs 2792/3000 ======> loss : 0.18101541013256844

epochs 2793/3000 ======> loss : 0.1809869529575026

epochs 2794/3000 ======> loss : 0.18095851319634315

epochs 2795/3000 ======> loss : 0.18093009083278216

epochs 2796/3000 ======> loss : 0.18090168585053204

epochs 2797/3000 ======> loss : 0.18087329823332526

epochs 2798/3000 ======> loss : 0.1808449279649147

epochs 2799/3000 ======> loss : 0.18081657502907303

epochs 2800/3000 ======> loss : 0.18078823940959282

epochs 2801/3000 ======> loss : 0.18075992109028716

epochs 2802/3000 ======> loss : 0.1807316200549887

epochs 2803/3000 ======> loss : 0.1807033362875505

epochs 2804/3000 ======> loss : 0.1806750697718451

epochs 2805/3000 ======> loss : 0.18064682049176525

epochs 2806/3000 ======> loss : 0.1806185884312235

epochs 2807/3000 ======> loss : 0.18059037357415214

epochs 2808/3000 ======> loss : 0.1805621759045033

epochs 2809/3000 ======> loss : 0.1805339954062492

epochs 2810/3000 ======> loss : 0.18050583206338144

e

# 5 - faisons quelques prédictions

In [65]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great','bad','great great', 'great great great', 'great great great bad']:
    prediction = predict_tweet(tweet, freqs, theta)
    if prediction > 0.5:
        print("{} --> {:.3f} --> positif".format(tweet,prediction[0,0]))
    else:
        print("{} --> {:.3f} --> negatif".format(tweet,prediction[0,0]))

I am happy --> 0.528 --> positif
I am bad --> 0.493 --> negatif
this movie should have been great. --> 0.523 --> positif
great --> 0.523 --> positif
bad --> 0.493 --> negatif
great great --> 0.546 --> positif
great great great --> 0.569 --> positif
great great great bad --> 0.561 --> positif


# 6 - evaluons notre modèle

In [52]:
def test_logistic_regression(x_test, y_test, freqs, theta):
    """
    Input: 
        x_test: a list of tweets
        y_test: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    
    # liste des prédictions de notre modèle
    y_hat = []
    
    for tweet in x_test:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0)

    #
    accuracy = sum((np.array(y_hat) == test_y.reshape(test_y.shape[0],)).astype(int))/test_y.shape[0]
    
    return accuracy

In [55]:
accuracy = test_logistic_regression(x_test, y_test, freqs, theta)
print(f"Logistic regression model's accuracy = {accuracy:.4f}")

Logistic regression model's accuracy = 0.9945


# nous avons une accuracy de 0.9945 ce qui est très bien 